<a href="https://colab.research.google.com/github/MineCamci/My_Project_For_Data_Science/blob/main/CRM_Analysis_RFM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##################################
#1.İŞ PROBLEMİ (BUSINESS PROBLEM)
###################################

#İngiltere merkezli online bir satış mağazasının Online Retail II isimli Veri Setini inceleyeceğiz.



#İÇERİK BİLGİLERİ: DEĞİŞKENLER;

#Invoice: Fatura numarası. Nominal. Her işleme benzersiz olarak atanan 6 haneli bir tam sayı. Bu kodun 'C' harfiyle başlaması iptal anlamına gelir.
#StockCode: Ürün (adet) kodu. Nominal. Her farklı ürüne benzersiz şekilde atanan 5 haneli tam sayı.
#Description: Ürün (öğe) adı. Nominal.
#Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
#InvoiceDate: Fatura tarihi ve saati. Sayısal. Bir işlemin oluşturulduğu gün ve saat.
#Price: Birim fiyat. Sayısal. Sterlin (£) cinsinden birim başına ürün fiyatı.
#Customer ID: Müşteri numarası. Nominal. Her müşteriye özel olarak atanan 5 haneli tam sayı.
#Country: Ülke adı. Nominal. Müşterinin ikamet ettiği ülkenin adı.

In [ ]:
######################################
#2. VERİ ANLAMA (DATA UNDERSTANDING
######################################
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format',lambda x: '%.3f' %x)
df = pd.read_csv("online_retail_09_10.csv") #excel formunda olursa ("dataset_name", sheet_name = "Year 2009-2010")
df = df.copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.250,13085.000,United Kingdom


In [ ]:
df.shape

(525461, 8)

In [ ]:
df.isnull().sum() #Toplam kaç boş değer var

InvoiceNo           0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     107927
Country             0
dtype: int64

In [ ]:
#essiz urun sayısı nedir?
df["Description"].nunique()

4681

In [ ]:
#Her bir üründen kaçar tane satıldı?
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

In [ ]:
#Description'a göre veriyi kır hangi üründen toplam kaçar tane sipariş verilmiş adet olarak.
#df.groupby("Description")["Quantity"].sum().head()
df.groupby("Description").agg({"Quantity": "sum"}).head() #üstteki gibi de olur negatif değerler çıktı küçükten büyüğe sıralayalım

,Quantity
Description,
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690
4 PURPLE FLOCK DINNER CANDLES,207
ANIMAL STICKERS,385
BLACK PIRATE TREASURE CHEST,57


In [ ]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity",ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [ ]:
#nunique number unique demek toplam eşsiz sayısı
df["InvoiceNo"].nunique() #Toplam kaç tane fatura kesilmiş?

28816

In [ ]:
df["InvoiceNo"].value_counts() #Length değeri nunique oluyor

537434     675
538071     652
537638     601
537237     597
536876     593
          ... 
523981       1
523984       1
523985       1
C523988      1
C516462      1
Name: InvoiceNo, Length: 28816, dtype: int64

In [ ]:
df["TotalPrice"] = df["Quantity"]*df["UnitPrice"]
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.250,13085.000,United Kingdom,30.000


In [ ]:
df.groupby("InvoiceNo").agg({"TotalPrice": "sum"}).head()

,TotalPrice
InvoiceNo,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


In [ ]:
########################################
#3.VERİ HAZIRLAMA (DATA PREPARATION)
########################################
df.shape
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     107927
Country             0
TotalPrice          0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(417534, 9)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.000,12.759,101.220,-9360.000,2.000,4.000,12.000,19152.000
UnitPrice,417534.000,3.888,71.132,0.000,1.250,1.950,3.750,25111.090
CustomerID,417534.000,15360.645,1680.811,12346.000,13983.000,15311.000,16799.000,18287.000
TotalPrice,417534.000,19.994,99.916,-25111.090,4.250,11.250,19.350,15818.400


In [ ]:
#İade edilenleri veri setinden çıkaralım "C" olanlar iadeyi belirtiyordu
df = df[~df["InvoiceNo"].str.contains("C", na=False)]
df.head() #"C" dışındakileri getir dedik çünkü iadeler eksi değer gösteriyordu yapıyı bozdu

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.250,13085.000,United Kingdom,30.000


In [ ]:
#########################################
#4.RFM METRIKLERINI HESAPLAMAK
#########################################

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.250,13085.000,United Kingdom,30.000


In [ ]:
df["InvoiceDate"].max()

'9/9/2010 20:03'

In [ ]:
type(df["InvoiceDate"].max())

str

In [ ]:
type("InvoiceDate")

str

In [ ]:
import datetime as dt
today_date = dt.datetime(2010, 12, 11)

In [ ]:
type(today_date)

datetime.datetime

In [ ]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format = '%m/%d/%Y %H:%M')

In [ ]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [ ]:
rfm = df.groupby("CustomerID").agg({"InvoiceDate":lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     "InvoiceNo": lambda InvoiceNo: InvoiceNo.nunique(),
                                     "TotalPrice": lambda TotalPrice: TotalPrice.sum()})
rfm

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930
...,...,...,...
18283.000,18,6,641.770
18284.000,67,1,461.680
18285.000,296,1,427.000


In [ ]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [ ]:
rfm.describe().T #Monetary min 0 çıktığı için düzenleme yapalım

,count,mean,std,min,25%,50%,75%,max
Recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
Frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
Monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [ ]:
rfm = rfm[rfm["Monetary"]>0]

In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
Frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
Monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [ ]:
rfm.shape

(4312, 3)

In [ ]:
#########################################
#5.RFM SKORLARININ HESAPLANMASI (CALCULATING RFM SCORES)
#########################################

#Recency ters Frequency ve Monetary düz bir şekilde 1-5 e kadar skor değeri olacak 1 en az 5 en yüksek

In [ ]:
  #qcut fonksiyonu için örnek:
  #0-20, 20-40 40-60 60-80 80-100  0-100 arası sayıları 5 e böl dedim etiketlerde (labels=) 5 4 3 2 1 olsun dersem böldüğüne karşılık sırayla koyar
  #5     4      3     2      1

In [ ]:
rfm["Recency_Score"] = pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
rfm.head()

<ipython-input-92-b2e5dbf96a89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["Recency_Score"] = pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])


,Recency,Frequency,Monetary,Recency_Score
CustomerID,,,,
12346.000,165,11,372.860,2
12347.000,3,2,1323.320,5
12348.000,74,1,222.160,2
12349.000,43,3,2671.140,3
12351.000,11,1,300.930,5


In [ ]:
rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'],5,labels=[1,2,3,4,5])
rfm.head(10)

<ipython-input-93-792582263f28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'],5,labels=[1,2,3,4,5])


,Recency,Frequency,Monetary,Recency_Score,Monetary_Score
CustomerID,,,,,
12346.000,165,11,372.860,2,2
12347.000,3,2,1323.320,5,4
12348.000,74,1,222.160,2,1
12349.000,43,3,2671.140,3,5
12351.000,11,1,300.930,5,2
12352.000,11,2,343.800,5,2
12353.000,44,1,317.760,3,2
12355.000,203,1,488.210,1,2
12356.000,16,3,3562.250,4,5


In [ ]:
rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])
rfm.head()

<ipython-input-94-a5ba1b63bc09>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])


,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score
CustomerID,,,,,,
12346.000,165,11,372.860,2,2,5
12347.000,3,2,1323.320,5,4,2
12348.000,74,1,222.160,2,1,1
12349.000,43,3,2671.140,3,5,3
12351.000,11,1,300.930,5,2,1


In [ ]:
rfm["RFM_SCORE"] = (rfm["Recency_Score"].astype(str)+
                    rfm["Frequency_Score"].astype(str))
rfm

<ipython-input-95-09f7133fcfe9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["RFM_SCORE"] = (rfm["Recency_Score"].astype(str)+


,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score,RFM_SCORE
CustomerID,,,,,,,
12346.000,165,11,372.860,2,2,5,25
12347.000,3,2,1323.320,5,4,2,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,5,3,33
12351.000,11,1,300.930,5,2,1,51
...,...,...,...,...,...,...,...
18283.000,18,6,641.770,4,3,5,45
18284.000,67,1,461.680,3,2,2,32
18285.000,296,1,427.000,1,2,2,12


In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
Frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
Monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [ ]:
rfm[rfm["RFM_SCORE"] == "55"]

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score,RFM_SCORE
CustomerID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
...,...,...,...,...,...,...,...
18225.000,1,15,7545.140,5,5,5,55
18226.000,14,15,6650.830,5,5,5,55
18229.000,2,10,3526.810,5,5,5,55


In [ ]:
rfm[rfm["RFM_SCORE"] == "11"]

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score,RFM_SCORE
CustomerID,,,,,,,
12355.000,203,1,488.210,1,2,1,11
12362.000,374,1,130.000,1,1,1,11
12366.000,269,1,500.240,1,2,1,11
12368.000,264,1,917.700,1,3,1,11
12378.000,198,1,1407.700,1,4,1,11
...,...,...,...,...,...,...,...
15928.000,292,1,293.530,1,2,1,11
15929.000,280,1,594.000,1,3,1,11
15941.000,273,1,405.000,1,2,1,11


In [ ]:
#######################################
# 6. SEGMENTLERIN ISIMLENDIRILMESI
#######################################
seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }


In [ ]:
rfm["SEGMENT"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)

<ipython-input-100-7c04a4544b2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["SEGMENT"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)


In [ ]:
rfm[["SEGMENT","Recency","Frequency","Monetary"]].groupby("SEGMENT").agg(["mean","count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
SEGMENT                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [ ]:
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["SEGMENT"] == "new_customers"].index

In [ ]:
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

In [ ]:
new_df.head()

,new_customer_id
0,12351
1,12385
2,12386
3,12427
4,12441


In [ ]:
new_df.to_csv("new_customer.csv")

In [ ]:
rfm.to_csv("rfm.csv")

In [ ]:
##########################################
#7.TÜM SÜRECİN FONKSİYONLAŞTIRILMASI
##########################################

#Yapılan tüm işlemleri tek fonskiyonda topladık ve istersek csv dosyası olarak çıktı alabilecek şekilde düzenledik.

def create_rfm(dataframe, csv = False ):
  #VERIYI HAZIRLAMA
  dataframe["TotalPrice"] = dataframe["Quantity"]*dataframe["UnitPrice"]
  dataframe = dataframe[~dataframe["InvoiceNo"].str.contains("C", na=False)]


  #RFM METRIKLERININ HESAPLANMASI
  today_date = dt.datetime(2010, 12, 11)
  rfm = dataframe.groupby("CustomerID").agg({"InvoiceDate":lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     "InvoiceNo": lambda InvoiceNo: InvoiceNo.nunique(),
                                     "TotalPrice": lambda TotalPrice: TotalPrice.sum()})
  rfm.columns = ['Recency','Frequency','Monetary']
  rfm = rfm[(rfm['Monetary'] > 0 )]

  #RFM SKORLARININ HESAPLANMASI
  rfm["Recency_Score"] = pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
  rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])
  rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'],5,labels=[1,2,3,4,5])

  #cltv_df skorları kategorik değere dönüştürülüp df'e eklendi
  rfm["RFM_SCORE"] = (rfm["Recency_Score"].astype(str) +
                      rfm["Frequency_Score"].astype(str))

  #SEGMENTLERIN ISIMLENDIRILMESI
  seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

  rfm["SEGMENT"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)
  rfm = rfm[["Recency","Frequency","Monetary","SEGMENT"]]
  rfm.index = rfm.index.astype(int)

  if csv:
    rfm.to_csv("rfm.csv")

  return rfm

In [ ]:
df = df.copy()

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [ ]:
rfm_new = create_rfm(df, csv=True)
rfm_new.head()

,Recency,Frequency,Monetary,SEGMENT
CustomerID,,,,
12346,165,11,372.860,cant_loose
12347,3,2,1323.320,potential_loyalists
12348,74,1,222.160,hibernating
12349,43,3,2671.140,need_attention
12351,11,1,300.930,new_customers
